In [2]:
import pymupdf4llm
import re
from langchain.transformers import pipeline

def convert_pdf_to_markdown(file_path):
    return pymupdf4llm.to_markdown(file_path)

def is_table(text):
    return '|' in text and '-|-' in text

def convert_table_to_text(table_md):
    # WikiBio 모델을 사용하여 표를 텍스트로 변환
    generator = pipeline('text2text-generation', model='facebook/bart-large-xsum-samsum')
    
    # 표 마크다운을 간단한 텍스트로 변환
    table_text = table_md.replace('|', ' ').replace('\n', ' ')
    
    # WikiBio 모델을 사용하여 텍스트 생성
    result = generator(table_text, max_length=150, min_length=30, do_sample=True)
    
    return result[0]['generated_text']

def process_markdown(md_text):
    lines = md_text.split('\n')
    processed_text = []
    current_table = []
    in_table = False

    for line in lines:
        if is_table(line):
            if not in_table:
                in_table = True
            current_table.append(line)
        else:
            if in_table:
                # 표 처리 완료, 텍스트로 변환
                table_text = convert_table_to_text('\n'.join(current_table))
                processed_text.append(table_text)
                current_table = []
                in_table = False
            processed_text.append(line)

    if current_table:
        # 마지막 표 처리
        table_text = convert_table_to_text('\n'.join(current_table))
        processed_text.append(table_text)

    return '\n'.join(processed_text)

def pdf_to_text_with_table_processing(file_path):
    md_text = convert_pdf_to_markdown(file_path)
    processed_text = process_markdown(md_text)
    return processed_text

# 사용 예
file_path = './open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'
result = pdf_to_text_with_table_processing(file_path)
print(result)

변환 완료: C:\Users\wjdwn\OneDrive\바탕 화면\Work\Dacon\재정정보 AI 검색 알고리즘 경진대회\output.txt
